In [1]:
# import dependencies

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import psycopg2
from sqlalchemy import create_engine


# may delete or move these these once model is more defined
#  moved down to split section-> from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
# create database connection
conn = psycopg2.connect(
    user = 'postgres',
    password = 'FinalProjectDB',
    host = 'finalprojectdb.chf41rwaufvy.us-east-1.rds.amazonaws.com',
    port = 5432,
    database = 'FinalProjectDB')

In [3]:
# import data and create dataframe
df_un = pd.read_sql_query("select * from \"combo_prod\"", conn)
pd.set_option('display.expand_frame_repr', False)
df_un.head(30)

,CASENUM,VEH_NO,STRATUM,VE_FORMS,NUMOCCS,NUMOCCSNAME,MONTHNAME,HARM_EV,HARM_EVNAME,MAN_COLL,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
0,'202002126769,2,10,2,2,2,January,12,Motor Vehicle In-Transport,7,...,1,2,5,12,7,0,1,1,2,2
1,'202002126774,1,9,2,1,1,January,12,Motor Vehicle In-Transport,6,...,1,2,5,12,6,0,1,1,2,2
2,'202002126774,2,9,2,5,5,January,12,Motor Vehicle In-Transport,6,...,1,2,5,12,6,0,1,1,2,2
3,'202002126779,1,10,1,1,1,January,38,Fence,0,...,1,2,5,38,0,0,1,1,2,2
4,'202002126780,1,9,2,1,1,January,12,Motor Vehicle In-Transport,1,...,0,10,5,12,1,0,1,3,10,2
5,'202002126780,2,9,2,2,2,January,12,Motor Vehicle In-Transport,1,...,0,10,5,12,1,0,1,3,10,2
6,'202002126790,1,10,1,1,1,January,34,Ditch,0,...,1,2,5,34,0,0,1,1,2,2
7,'202002126794,1,5,1,2,2,January,31,"Post, Pole or Other Supports",0,...,0,2,6,31,0,0,1,1,2,2
8,'202002126801,1,7,2,3,3,January,12,Motor Vehicle In-Transport,1,...,0,2,6,12,1,0,1,3,2,2
9,'202002126801,2,7,2,2,2,January,12,Motor Vehicle In-Transport,1,...,0,2,6,12,1,0,1,3,2,2


In [4]:
#filter dataframe to remove unknows categories
df = df_un[df_un['MAX_SEV'] < 5]
df.head(30)

,CASENUM,VEH_NO,STRATUM,VE_FORMS,NUMOCCS,NUMOCCSNAME,MONTHNAME,HARM_EV,HARM_EVNAME,MAN_COLL,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
0,'202002126769,2,10,2,2,2,January,12,Motor Vehicle In-Transport,7,...,1,2,5,12,7,0,1,1,2,2
1,'202002126774,1,9,2,1,1,January,12,Motor Vehicle In-Transport,6,...,1,2,5,12,6,0,1,1,2,2
2,'202002126774,2,9,2,5,5,January,12,Motor Vehicle In-Transport,6,...,1,2,5,12,6,0,1,1,2,2
3,'202002126779,1,10,1,1,1,January,38,Fence,0,...,1,2,5,38,0,0,1,1,2,2
4,'202002126780,1,9,2,1,1,January,12,Motor Vehicle In-Transport,1,...,0,10,5,12,1,0,1,3,10,2
5,'202002126780,2,9,2,2,2,January,12,Motor Vehicle In-Transport,1,...,0,10,5,12,1,0,1,3,10,2
6,'202002126790,1,10,1,1,1,January,34,Ditch,0,...,1,2,5,34,0,0,1,1,2,2
7,'202002126794,1,5,1,2,2,January,31,"Post, Pole or Other Supports",0,...,0,2,6,31,0,0,1,1,2,2
8,'202002126801,1,7,2,3,3,January,12,Motor Vehicle In-Transport,1,...,0,2,6,12,1,0,1,3,2,2
9,'202002126801,2,7,2,2,2,January,12,Motor Vehicle In-Transport,1,...,0,2,6,12,1,0,1,3,2,2


In [5]:
#close data source connection
conn.close()

In [6]:
# separate targets

y = df["MAX_SEV"]
y.tail(5)


94343    1
94344    1
94345    1
94346    0
94347    0
Name: MAX_SEV, dtype: int64

In [7]:
#create resulting features dataframe
X_df = df[["VEH_NO","VE_FORMS","NUMOCCS","HARM_EV","MAN_COLL","UNITTYPE","HIT_RUN","MAKE","MODEL","MAK_MOD","BODY_TYP","MOD_YEAR","TOW_VEH","J_KNIFE","TRAV_SP","ROLLOVER","ROLINLOC","DEFORMED","TOWED","M_HARM","VEH_ALCH","FIRE_EXP","DR_PRES","DR_ZIP","SPEEDREL","VTRAFWAY","VNUM_LAN","VALIGN","VPROFILE","VSURCOND","VTRAFCON","VTCONT_F","P_CRASH1","P_CRASH2","P_CRASH3","PCRASH4","PCRASH5","ACC_TYPE","VPICMAKE","VPICMODEL","VPICBODYCLASS","ICFINALBODY","GVWR_FROM","GVWR_TO","REGION","URBANICITY","MONTH","YEAR","DAY_WEEK","ALCOHOL","RELJCT1","RELJCT2","TYP_INT","WRK_ZONE","REL_ROAD","LGT_COND","INT_HWY","WEATHER"]]
X_df.tail(5)

,VEH_NO,VE_FORMS,NUMOCCS,HARM_EV,MAN_COLL,UNITTYPE,HIT_RUN,MAKE,MODEL,MAK_MOD,...,DAY_WEEK,ALCOHOL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,INT_HWY,WEATHER
94343,1,1,1,24,0,1,0,35,43,35043,...,6,2,1,5,1,0,4,1,1,10
94344,1,2,1,12,1,1,0,2,403,2403,...,7,2,0,3,3,0,1,1,0,2
94345,2,2,1,12,1,1,0,84,881,84881,...,7,2,0,3,3,0,1,1,0,2
94346,1,1,1,46,0,1,0,35,39,35039,...,4,2,0,6,1,0,1,1,0,10
94347,1,2,4,12,7,1,0,49,32,49032,...,5,2,0,1,1,0,1,1,1,2


In [8]:
# partition data into training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, 
                                                    y, random_state =1,test_size = .2)
X_train.shape

(74485, 58)

In [9]:
#Resample the training data to account for variations in the sampe
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [10]:
# create the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)

In [11]:
# train the model *can toggle between resampled data
classifier.fit(X_resampled, y_resampled)
#classifier.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
# make preditions with the model
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,2,2
1,2,0
2,0,0
3,0,0
4,0,0
5,0,0
6,1,1
7,1,1
8,0,0
9,0,1


In [13]:
# export results to a csv file
df_results=pd.DataFrame(results)
df_results.to_csv('results.csv')

In [14]:
# validate the model

In [15]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5505853291805392


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[6958  891  415  246    7]
 [1976 1498  684  292    8]
 [1014  683 1083  397   20]
 [ 453  387  496  638   32]
 [  60   48   84  176   76]]


In [17]:
# export confusion matrix to a csv file
df_matrix=pd.DataFrame(matrix)
df_matrix.to_csv('matrix.csv')

In [18]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.67      0.82      0.73      8517
           1       0.43      0.34      0.38      4458
           2       0.39      0.34      0.36      3197
           3       0.36      0.32      0.34      2006
           4       0.53      0.17      0.26       444

    accuracy                           0.55     18622
   macro avg       0.48      0.40      0.41     18622
weighted avg       0.53      0.55      0.53     18622



In [19]:
#